In [1]:
#最低限必要な情報:物件名・住所・間取り・家賃・階数
#検索エリア：23区、2K以上、家賃管理費込み22万円未満の物件情報

In [2]:
# ライブラリのインポート
from bs4 import BeautifulSoup
import requests
import csv
import time

# 物件情報を抽出する関数
def scrape_property_data(soup):
    property_data = []
    property_panels = soup.find_all("div", class_="PropertyPanel_propertyPanel__MqCpF")
    for panel in property_panels:
        title = panel.find("div", class_="PropertyPanelBuilding_buildingTitle__NbWmb").text.strip().split("】")[1].strip()  # タイトルの抽出
        address = panel.find("p", class_="is-mt5").text.strip()  # 住所の抽出
        access_info = panel.find_all("div", class_="PropertyPanelBuilding_buildingInformationSection__AMRsh")[0]
        access = "、".join([p.text for p in access_info.find_all("p") if not p.get("class")])  # アクセス情報の抽出
        age = panel.find_all("div", class_="PropertyPanelBuilding_buildingInformationSection__AMRsh")[1].find("p").text.strip()  # 築年数の抽出
        floor = panel.find_all("div", class_="PropertyPanelBuilding_buildingInformationSection__AMRsh")[1].find_all("p")[1].text.strip()  # 建物の階数の抽出
        fee = panel.find("div", class_="PropertyPanelRoom_rentPrice__HO4Jp").text.split("(")[0].strip()  # 賃料の抽出
        management_fee = panel.find("div", class_="PropertyPanelRoom_rentPrice__HO4Jp").text.split("(")[1].replace(")", "").strip() if len(panel.find("div", class_="PropertyPanelRoom_rentPrice__HO4Jp").text.split("(")) > 1 else ""  # 管理費の抽出
        if "管理費" in management_fee:
            management_fee = management_fee.replace("管理費", "").strip()
        
        # 敷金と礼金の抽出
        initial_prices = panel.find("div", class_="PropertyPanelRoom_initialPrices__naYEA")
        if initial_prices:
            price_elements = initial_prices.find_all("li")
            deposit = price_elements[0].text.strip() if price_elements else ""
            gratuity = price_elements[1].text.strip() if len(price_elements) > 1 else ""
        else:
            deposit, gratuity = "", ""
        
        room_info_element = panel.find("span", class_="is-ml5")
        if room_info_element:
            room_info_text = room_info_element.get_text().split("/")
            floor_plan = room_info_text[1].strip() if len(room_info_text) > 1 else ""  # 間取りの抽出
            area = room_info_text[2].strip() if len(room_info_text) > 2 else ""  # 面積の抽出
        else:
            floor_plan, area = "", ""

        # 仲介手数料の抽出
        RoomItem = panel.select("a.PropertyPanelRoom_roomItem__3bVhC")[0].select("div.PropertyPanelRoom_roomItemRightContent__UzfRX")[0].select("div")[2]
        if "PropertyPanelRoom_isRed__HXtHO" in str(RoomItem):
            charge = RoomItem.select("span.PropertyPanelRoom_isRed__HXtHO")[0].text
            charge = "0円" if "無料" in charge else charge
        else:
            charge = RoomItem.select("span")[1].text

        property_data.append([title, address, access, age, floor, fee, management_fee, deposit, gratuity, floor_plan, area, charge])
    return property_data

def main():
    base_url = "https://airdoor.jp/list?si=d-131016-131024-131032-131041-131059-131067-131075-131083-131091-131105-131113-131121-131130-131148-131156-131164-131172-131181-131199-131202-131211-131229-131237&ur=220000&iaf=1&fp=d-2_k-2_dk-2_ldk-3_k-3_dk-3_ldk-4_more&p={}"
    data_list = []

    page = 1
    while True:
        print(f"ページ {page} を取得中...")
        target_url = base_url.format(page)
        r = requests.get(target_url)

        if r.status_code != 200:
            print("最後のページまで取得しました。")
            break

        soup = BeautifulSoup(r.text, 'html.parser')
        property_data = scrape_property_data(soup)

        # 物件情報がない場合、ループを終了
        if not property_data:
            print(f"ページ {page} に物件情報がありません。スクレイピングを終了します。")
            break

        data_list.extend(property_data)
        page += 1
        time.sleep(1)  # 1秒間のウェイト

    # CSVファイルへの書き込み
    with open('airdoor_data.csv', 'w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Address', 'Access', 'Age', 'Floor', 'Fee', 'Management Fee', 'Deposit', 'Gratuity', 'Floor Plan', 'Area', 'Charge'])
        writer.writerows(data_list)

    print("スクレイピングが完了しました。")

if __name__ == "__main__":
    main()


ページ 1 を取得中...
ページ 2 を取得中...
ページ 3 を取得中...
ページ 4 を取得中...
ページ 5 を取得中...
ページ 6 を取得中...
ページ 7 を取得中...
ページ 8 を取得中...
ページ 9 を取得中...
ページ 10 を取得中...
ページ 11 を取得中...
ページ 12 を取得中...
ページ 13 を取得中...
ページ 14 を取得中...
ページ 15 を取得中...
ページ 16 を取得中...
ページ 17 を取得中...
ページ 18 を取得中...
ページ 19 を取得中...
ページ 20 を取得中...
ページ 21 を取得中...
ページ 22 を取得中...
ページ 23 を取得中...
ページ 24 を取得中...
ページ 25 を取得中...
ページ 26 を取得中...
ページ 27 を取得中...
ページ 28 を取得中...
ページ 29 を取得中...
ページ 30 を取得中...
ページ 31 を取得中...
ページ 32 を取得中...
ページ 33 を取得中...
ページ 34 を取得中...
ページ 35 を取得中...
ページ 36 を取得中...
ページ 36 に物件情報がありません。スクレイピングを終了します。
スクレイピングが完了しました。
